In [212]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port="5433",
    database="krankenhaus",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()


In [213]:
# ZENTRALE PARAMETER (einfach hier ändern)

ARZT_ID = 1
PFLEGEKRAFT_ID = 1
KRANKENHAUS_ID = 1
TODAY = '2025-12-12'
PATIENT_VORNAME = "Waldemar"
PATIENT_NACHNAME = "Schwarz"


def execute_print_query(cursor,query):
    cursor.execute(query)
    column_names = [desc[0] for desc in cur.description]
    print(column_names)
    for row in cur.fetchall():
        print(row)


Dr. Luc Thommen kommt zur Arbeit und möchte wissen, welche Patienten er heute betreut und wo sie sich befinden.

In [214]:
query = f"""
-- Zeigt alle heutigen Patienten eines Arztes inklusive Station und Event
SELECT
    p.patient_id,
    p.vorname || ' ' || p.nachname AS patient_name,
    s.name AS station,
    e.datum,
    e.beschreibung AS event_beschreibung
FROM arzt a
JOIN arzt_event ae ON a.arzt_id = ae.arzt_id
JOIN event e ON ae.event_id = e.event_id
JOIN behandlung b ON e.behandlung_id = b.behandlungs_id
JOIN patient p ON b.patient_id = p.patient_id
JOIN station s ON p.station_id = s.station_id
WHERE a.arzt_id = {ARZT_ID}
  AND e.datum = DATE '{TODAY}'
ORDER BY s.name, patient_name;
"""
execute_print_query(cur, query)


['patient_id', 'patient_name', 'station', 'datum', 'event_beschreibung']
(133, 'Gonçalo Huber', 'Chirurgie', datetime.date(2025, 12, 12), 'Medikamentengabe')
(120, 'Iva Blaser', 'Chirurgie', datetime.date(2025, 12, 12), 'Nachkontrolle')
(28, 'Maddalena Maier', 'Chirurgie', datetime.date(2025, 12, 12), 'Visite')
(4, 'Maja Aebi', 'Chirurgie', datetime.date(2025, 12, 12), 'Medikamentengabe')
(23, 'Waldemar Schwarz', 'Chirurgie', datetime.date(2025, 12, 12), 'Medikamentengabe')


Der Arzt erkennt, dass Tanja Bernasconi heute mehrfach auf seiner Liste auftaucht.
Bevor er das Zimmer betritt, möchte er sich einen Überblick verschaffen, welche Behandlungen sie bisher hatte und wann diese stattgefunden haben.

In [215]:
query = f"""
-- Zeigt alle bisherigen Behandlungen einer bestimmten Patientin (Patientendossier)
SELECT
    b.behandlungs_id,
    p.vorname || ' ' || p.nachname AS patient_name,
    b.beschreibung,
    b.abgeschlossen,
    b.start_datum,
    b.end_datum
FROM patient p
JOIN behandlung b ON p.patient_id = b.patient_id
WHERE p.vorname = '{PATIENT_VORNAME}'
  AND p.nachname = '{PATIENT_NACHNAME}'
ORDER BY b.start_datum;
"""
execute_print_query(cur, query)


['behandlungs_id', 'patient_name', 'beschreibung', 'abgeschlossen', 'start_datum', 'end_datum']
(3, 'Waldemar Schwarz', 'Atemwegsinfekt Behandlung', True, datetime.date(2025, 11, 13), datetime.date(2025, 12, 12))


Der Arzt hat sich einen Überblick über Tanja Bernasconi Krankengeschichte verschafft.
Jetzt möchte er wissen, welche Medikamente sie im Verlauf ihrer Behandlungen erhalten hat, um Wechselwirkungen zu vermeiden und die heutige Therapie richtig zu planen.

In [216]:
query = f"""
-- Zeigt alle Medikamente, die einer Patientin im Rahmen ihrer Behandlungen verabreicht wurden
SELECT DISTINCT
    m.medikament_id,
    m.name AS medikament_name,
    m.wirkstoff,
    e.datum AS event_datum,
    e.beschreibung AS event_typ
FROM patient p
JOIN behandlung b ON p.patient_id = b.patient_id
JOIN event e ON b.behandlungs_id = e.behandlung_id
JOIN event_medikament em ON e.event_id = em.event_id
JOIN medikament m ON em.medikament_id = m.medikament_id
WHERE p.vorname = '{PATIENT_VORNAME}'
  AND p.nachname = '{PATIENT_NACHNAME}'
ORDER BY e.datum, m.name;
"""
execute_print_query(cur, query)


['medikament_id', 'medikament_name', 'wirkstoff', 'event_datum', 'event_typ']
(4, 'Amoxicillin', 'Amoxicillin', datetime.date(2025, 12, 12), 'Medikamentengabe')
(1, 'Paracetamol', 'Paracetamol', datetime.date(2025, 12, 12), 'Medikamentengabe')


Nachdem der Arzt weiß, welche Behandlungen und Medikamente Tanja Bernasconi betreffen,
möchte er jetzt ganz konkret wissen, wo sie sich aktuell befindet, um persönlich bei ihr vorbeizugehen.

In [217]:
query = """
SELECT
    pk.vorname || ' ' || pk.nachname AS pflegekraft,
    p.patient_id,
    p.vorname || ' ' || p.nachname AS patient_name,
    s.name AS station
FROM pflegekraft pk
JOIN station s ON pk.station_id = s.station_id
JOIN patient p ON p.station_id = s.station_id
WHERE pk.pflege_id = %s
ORDER BY patient_name;
"""
execute_print_query(cur, cur.mogrify(query, (PFLEGEKRAFT_ID,)).decode())


['pflegekraft', 'patient_id', 'patient_name', 'station']


Die Pflegekraft kontrolliert, welche Events heute auf ihrer Station stattfinden.

In [218]:
query = """
SELECT
    e.event_id,
    e.uhrzeit,
    e.beschreibung,
    r.art AS raumart
FROM pflegekraft pk
JOIN station s ON pk.station_id = s.station_id
JOIN raum r ON r.station_id = s.station_id
JOIN event e ON e.raumid = r.raumnummer
WHERE pk.pflege_id = %s
  AND e.datum = %s
ORDER BY e.uhrzeit;
"""
execute_print_query(cur, cur.mogrify(query, (PFLEGEKRAFT_ID, DATUM)).decode())


['event_id', 'uhrzeit', 'beschreibung', 'raumart']


In [219]:
# Query 5: Zeigt alle Events des heutigen Tages,
# sortiert nach Raum und Uhrzeit (z. B. für Tagesplanung).

query = """
SELECT
    r.art AS raum_art,
    r.raumnummer,
    e.uhrzeit,
    p.vorname || ' ' || p.nachname AS patient_name
FROM event e
JOIN raum r ON e.raumid = r.raumnummer
JOIN behandlung b ON e.behandlung_id = b.behandlungs_id
JOIN patient p ON b.patient_id = p.patient_id
WHERE e.datum = CURRENT_DATE
ORDER BY r.raumnummer, e.uhrzeit;
"""

cur.execute(query)
columns = [desc[0] for desc in cur.description]
rows = cur.fetchall()

print(columns)
for row in rows:
    print(row)


['raum_art', 'raumnummer', 'uhrzeit', 'patient_name']
('Operationssaal', 1, datetime.time(7, 31, 11), 'Iva Blaser')
('Operationssaal', 1, datetime.time(20, 2, 15), 'Samanta Hartmann')
('Untersuchungsraum', 3, datetime.time(8, 35, 26), 'Christel Egger')
('Patientenzimmer', 4, datetime.time(1, 5, 24), 'Gonçalo Huber')
('Patientenzimmer', 4, datetime.time(20, 25, 24), 'Antonella Amrein')
('Patientenzimmer', 5, datetime.time(10, 22, 43), 'Branislav Seiler')
('Patientenzimmer', 6, datetime.time(5, 1, 11), 'Caspar Staub')
('Patientenzimmer', 6, datetime.time(14, 13, 25), 'Waldemar Schwarz')
('Patientenzimmer', 6, datetime.time(17, 36, 7), 'Selina Furrer')
('Patientenzimmer', 8, datetime.time(0, 34, 39), 'Jozef Stöckli')
('Patientenzimmer', 9, datetime.time(17, 53, 36), 'Maja Aebi')
('Aufenthaltsraum', 10, datetime.time(16, 20, 14), 'Maddalena Maier')
('Operationssaal', 11, datetime.time(9, 12, 43), 'Eveline Spörri')
('Untersuchungsraum', 12, datetime.time(4, 27, 4), 'Catarina Zürcher')
('Unt